In [1]:
import cv2
import os
import math
import mediapipe as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle

In [2]:
df=pd.read_csv('TrainData.csv')
asana_names=df.Asana

In [3]:
filename = 'classifier.sav'
classifier= pickle.load(open(filename, 'rb'))

filename = 'kNN.sav'
kNN= pickle.load(open(filename, 'rb'))

In [5]:
def find_mid(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return (a+b)/2

def distance(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return ( ((a[0]-b[0])**2) + ((a[1]-b[1])**2) + ((a[2]-b[2])**2) ) **0.5

def magnitude(a):
    a = np.array(a)
    return ((a[0]**2)+(a[1]**2)+(a[2]**2))**0.5

def get_line(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return a-b
         
def get_joints(landmarks):
#     joints = head(mid point of eyes), neck, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, mid hip, right hip, left knee, right knee, left ankle, right ankle
    joints=[]
    # print("landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x")
    # print(landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x)
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].z],
                            [landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].z])
                  )
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z])
                 )
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]))
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z])
                 )
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]))
    
    return joints

def get_body_lines(joints):
    body_lines=[]
    body_lines.append(get_line(joints[0],joints[1])) # head to neck
    body_lines.append(get_line(joints[1],joints[9])) # neck to hip
    body_lines.append(get_line(joints[8],joints[9])) # left_hip to mid_hip
    body_lines.append(get_line(joints[10],joints[9])) # right_hip to mid_hip
    body_lines.append(get_line(joints[8],joints[11])) # left_hip to left_knee
    body_lines.append(get_line(joints[10],joints[12])) # right_hip to right_knee
    body_lines.append(get_line(joints[11],joints[13])) # left_knee to left_ankle
    body_lines.append(get_line(joints[12],joints[14])) # right_knee to right_ankle
    body_lines.append(get_line(joints[2],joints[1])) # left_shoulder to neck
    body_lines.append(get_line(joints[3],joints[1])) # right_shoulder to neck
    body_lines.append(get_line(joints[2],joints[4])) # left_shoulder to left_elbow
    body_lines.append(get_line(joints[4],joints[6])) # left_elbow to left_wrist
    body_lines.append(get_line(joints[3],joints[5])) # right_shoulder to right_elbow
    body_lines.append(get_line(joints[5],joints[7])) # right_elbow to right_wrist
#     for i in range(0,len(joints)):
#         for j in range(i+1,len(joints)):
#             body_lines.append(get_line(joint[i],joints[j]))
    # print("body_lines")
    # print(len(body_lines))
    # print(body_lines)
    return body_lines

def get_body_angles(body_lines):
    body_angles=[]
    for i in range(0,len(body_lines)):
        for j in range(i+1,len(body_lines)):
            # print("i,j ")
            # print(i)
            # print(j)
            numerator=np.dot(body_lines[i],body_lines[j])
            # print("numerator")
            # print(numerator)
            denominator=magnitude(body_lines[i])*magnitude(body_lines[j])
            # print("denominator")
            # print(denominator)
            argument = max(-1.0, min(numerator / denominator, 1.0))
            body_angles.append(math.acos(argument))
            # print("No eerrrorr")
    return body_angles  

def calculate_features(image):
    output=[]
    # Checking if the image is empty or not
    if image is None:
        result = "Image is empty!!"
        return
    # image=image_resize(image)
    
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        results = pose.process(image)
        # print("results")
        # print(results)
        if not results.pose_landmarks:
            return
        try:
            landmarks = results.pose_landmarks.landmark
            # print("landmarks")
            # print(landmarks)
            output=[]
            joints=get_joints(landmarks)
            # print("joints")
            # print(joints)
#             output=output+joints
            # get_body_lines(joints)
            bodyLines=get_body_lines(joints)
            output=output+get_body_angles(bodyLines)
            # print("output")
            # print(output)
            return output
        except Exception as e:
            print("Error in calculateFetaure :")
            print(e)
            

In [19]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(r"1.mp4")
if (cap.isOpened()== False):
    print("Error opening video stream or file")

confidence=0

cv2.namedWindow('Mediapipe Feed', cv2.WINDOW_NORMAL)
# Set the initial window size
cv2.resizeWindow('Mediapipe Feed', 1280, 720) 

label_mapping = {0: 'Bhujangasana', 1: 'JanuSirasana', 2: 'Padmasana', 3: 'Savasana', 4: 'Tadasana', 5: 'Trikonasana'}
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cap.read()
        if ret:
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                test=[]+get_body_angles(get_body_lines(get_joints(landmarks)))
#                 print(test)
                asana=classifier.predict(np.array(test).reshape(1, -1))
                asana_name = label_mapping.get(asana[0], 'Unknown')
                confidence=sorted(classifier.predict_proba(np.array(test).reshape(1, -1))[0])[-1]
                cv2.putText(image, 'Asana = '+str(asana[0])+asana_name, 
                           (50, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0,255), 2, cv2.LINE_AA
                                )
                cv2.putText(image, 'Confidence = '+str((confidence*10000)//100)+'%', 
                           (10, 78), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0,255), 2, cv2.LINE_AA
                                )
            except Exception as e:
                print(e)
                pass

            # Render detections
            mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS)            
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(r"check1.mp4")
if (cap.isOpened()== False):
    print("Error opening video stream or file")

confidence=0
max_confidence=0
prev_confidence=0
results2=0
# Setup mediapipe instance
# similar_image=None
label_mapping = {0: 'Bhujangasana', 1: 'JanuSirasana', 2: 'Padmasana', 3: 'Savasana', 4: 'Tadasana', 5: 'Trikonasana'}
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cap.read()
        # if similar_image is None:
        #     similar_image=cv2.imread(r'none.jpg', cv2.IMREAD_COLOR)
        #     # similar_image=image_resize_small(similar_image)
        if ret:
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # image=image_resize(image)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                test=[]+get_body_angles(get_body_lines(get_joints(landmarks)))
#                 print(test)
                asana=classifier.predict(np.array(test).reshape(1, -1))
                asana_name = label_mapping.get(asana[0], 'Unknown')
                confidence=sorted(classifier.predict_proba(np.array(test).reshape(1, -1))[0])[-1]
                cv2.putText(image, 'Asana = '+asana_name, 
                           (20, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0,255), 2, cv2.LINE_AA
                                )
                cv2.putText(image, 'Confidence = '+str(((confidence*1000000)//100)/100)+'%', 
                           (20, 80), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0,255), 2, cv2.LINE_AA
                                )
                if(confidence>=0.75 and confidence>prev_confidence):
                    max_confidence=confidence
                    similar=kNN.kneighbors(X=np.array(test).reshape(1, -1), return_distance=True)[1][0][0]
                    ideal_features=df.iloc[similar]
                    curr_features=test
                    matrix=[]
                    matrix.append(ideal_features.feature_1-curr_features[0])
                    matrix.append(ideal_features.feature_78-curr_features[77])
                    matrix.append(ideal_features.feature_84-curr_features[83])
                    matrix.append(ideal_features.feature_86-curr_features[85])
                    matrix.append(ideal_features.feature_91-curr_features[90])
                    matrix.append(ideal_features.feature_16-curr_features[15])
                    matrix.append(ideal_features.feature_17-curr_features[16])
                    matrix.append(ideal_features.feature_48-curr_features[47])
                    matrix.append(ideal_features.feature_57-curr_features[56])
#                     matrix=matrix/math.pi
                    advice=[]
                    for diff in matrix:
                        diff=diff*180/math.pi
#                         advice.append(str(diff))
                        if(abs(diff)<=13.9):
                            advice.append("Correct")
                        elif(diff<0):
                            advice.append('Open more')
                        else:
                            advice.append('Bend more')
#                         else:
#                             advice.append(str(diff))
                        
#                     print(features)
                    
                    asana=asana_names[similar]
                    # file_name=file_names[similar]
                    # similar_image=cv2.imread(os.path.join('Images', asana, file_name), cv2.IMREAD_COLOR)
                    # similar_image=image_resize_small(similar_image)
                
                if confidence>=0.75 and len(np.unique(advice))==1 and advice[0]=='Correct':
                    cv2.putText(image, 'Correct pose', 
                           (470, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2, cv2.LINE_AA
                                )
                elif max_confidence>0 :
#                     cv2.putText(image, 'Neck = '+advice[0], 
#                            (510, 50), 
#                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
#                                 )
                    cv2.putText(image, 'left_shoulder= '+advice[1], 
                           (800, 190), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    cv2.putText(image, 'right_shoulder = '+advice[2], 
                           (100, 190), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    cv2.putText(image, 'left_elbow = '+advice[3], 
                           (820, 300), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    cv2.putText(image, 'right_elbow = '+advice[4], 
                           (90, 300), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    cv2.putText(image, 'left_hip = '+advice[5], 
                           (800, 400), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    cv2.putText(image, 'right_hip = '+advice[6], 
                           (100, 400), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    cv2.putText(image, 'left_knee = '+advice[7], 
                           (800, 500), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    cv2.putText(image, 'right_knee = '+advice[8], 
                           (100, 500), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 1, cv2.LINE_AA
                                )
                    prev_confidence=confidence
                    
            except Exception as e:
#                 print(e)
                pass

            # Render detections
            mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS)            
            
            cv2.imshow('Mediapipe Feed', image)
            # cv2.imshow('Similar_image', similar_image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()